# Hunyuan3D 2.0 - Furniture to 3D Models

Convert furniture images (with transparent backgrounds) into 3D GLB models.

**Note:** If Hunyuan3D-2 setup is complex, consider using the `simple_3d_generator.ipynb` notebook instead, which uses Shap-E or TripoSR.

**Setup:** Runtime → Change runtime type → GPU (T4 or better)

In [ ]:
# Install dependencies
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate diffusers
!pip install -q trimesh pygltflib pillow numpy gradio

# Clone Hunyuan3D
!git clone -q https://github.com/Tencent-Hunyuan/Hunyuan3D-2.git
!cd Hunyuan3D-2 && pip install -q -r requirements.txt

import sys
sys.path.append('/content/Hunyuan3D-2')
print("✅ Dependencies installed")

In [ ]:
# Setup and download model
import os
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')
INPUT_DIR = '/content/drive/MyDrive/furniture_images/background_removed'
OUTPUT_DIR = '/content/drive/MyDrive/furniture_images/3d_models'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check what's in the Hunyuan3D-2 directory
!ls -la /content/Hunyuan3D-2/

# Download model weights from Hugging Face
print("Downloading Hunyuan3D model weights...")
!mkdir -p /content/models
# Using the official Hugging Face model
!cd /content/models && wget -c https://huggingface.co/Tencent/Hunyuan3D-2/resolve/main/hunyuan3d-2.0.ckpt

print("✅ Setup complete")

In [ ]:
# 3D generation interface
import torch
import numpy as np
from PIL import Image
import gradio as gr
from pathlib import Path

# First, let's check the repository structure
import sys
sys.path.append('/content/Hunyuan3D-2')

# Try to find the correct import path
try:
    # Check if there's an app.py or main inference script
    !cd /content/Hunyuan3D-2 && find . -name "*.py" | grep -E "(app|main|inference)" | head -10
except:
    pass

# For now, let's use a simpler approach with the actual Hunyuan3D API
# Based on the repository, it seems to use a different structure

def generate_3d_model(image, quality="Standard"):
    """
    Generate 3D model from image using Hunyuan3D-2
    """
    if image is None:
        return None, None, "Please upload an image"
    
    # Save input image temporarily
    temp_input = "/content/temp_input.png"
    image.save(temp_input)
    
    # Quality settings
    quality_map = {
        "Fast": "--steps 30",
        "Standard": "--steps 50", 
        "High": "--steps 100"
    }
    
    try:
        # Run Hunyuan3D command
        output_name = f"model_{len(os.listdir(OUTPUT_DIR))}"
        output_path = f"{OUTPUT_DIR}/{output_name}.glb"
        
        # Execute the model (adjust command based on actual repo structure)
        !cd /content/Hunyuan3D-2 && python app.py \
            --input {temp_input} \
            --output {output_path} \
            --model_path /content/models/hunyuan3d-2.0.ckpt \
            {quality_map[quality]}
        
        # Create a preview image
        preview = image  # For now, use input as preview
        
        return preview, output_path, f"✅ Saved to {output_path}"
        
    except Exception as e:
        return None, None, f"Error: {str(e)}"

# Create Gradio interface
with gr.Blocks(title="Hunyuan3D") as app:
    gr.Markdown("""
    # 🎯 Furniture → 3D Model
    
    Convert furniture images with transparent backgrounds into 3D models.
    """)
    
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(type="pil", label="Upload Furniture Image")
            quality = gr.Radio(["Fast", "Standard", "High"], value="Standard", label="Quality")
            generate_btn = gr.Button("Generate 3D Model", variant="primary")
            status = gr.Textbox(label="Status")
            
        with gr.Column():
            preview = gr.Image(label="Preview")
            download = gr.File(label="Download GLB")
    
    generate_btn.click(
        generate_3d_model,
        inputs=[input_img, quality],
        outputs=[preview, download, status]
    )
    
    # Load examples
    import glob
    examples = glob.glob(f"{INPUT_DIR}/*.png")[:3]
    if examples:
        gr.Examples(examples=[[ex] for ex in examples], inputs=[input_img])

# First, let's check what files are available in the repo
print("Checking Hunyuan3D-2 repository structure...")
!cd /content/Hunyuan3D-2 && ls -la

app.launch(share=True)

In [ ]:
# Batch processing (optional)
def batch_convert(folder_path=INPUT_DIR, quality="Standard"):
    import glob
    from tqdm import tqdm
    
    images = glob.glob(f"{folder_path}/*.png")
    print(f"Found {len(images)} images")
    
    for img_path in tqdm(images):
        try:
            img = Image.open(img_path)
            _, path, _ = generate_3d(img, quality)
            print(f"✓ {os.path.basename(img_path)} → {os.path.basename(path)}")
        except Exception as e:
            print(f"✗ {os.path.basename(img_path)}: {e}")

# Run batch: batch_convert(quality="Fast")